# PyTorch Basics Notebook
### Introduction to Tensors, Datasets, DataLoaders, CNNs, and U‑Net Building Blocks

## Import Required Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt


## PyTorch Tensors

In [2]:
# Creating tensors
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.randn(3)  # random tensor

print(x)
print(y)

# Tensor operations
print('Addition:', x + y)
print('Mean:', x.mean())


tensor([1., 2., 3.])
tensor([ 1.7628,  0.8428, -0.2014])
Addition: tensor([2.7628, 2.8428, 2.7986])
Mean: tensor(2.)


## Autograd Basics

In [3]:
# Enable gradient tracking
a = torch.tensor([2.0, 3.0], requires_grad=True)
b = (a * a).sum()
b.backward()
print(a.grad)  # derivative of x^2 is 2x


tensor([4., 6.])


## Custom PyTorch Dataset

We simulate MRI-like slices using random arrays just for practice.

In [4]:
class RandomMRIDataset(Dataset):
    def __init__(self, length=100):
        self.length = length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = torch.randn(1, 64, 64)      # fake MRI slice
        mask = (torch.randn(1, 64, 64) > 0).float()  # fake mask
        return image, mask

dataset = RandomMRIDataset()
img, msk = dataset[0]
print(dataset[1])
# print(dataset[1])


(tensor([[[-0.4522, -0.3135,  0.0866,  ..., -0.1870, -1.1722, -0.8596],
         [ 0.1491,  0.6011, -1.1154,  ..., -1.6550,  1.0204, -0.3005],
         [ 0.1871,  1.1978, -0.7697,  ..., -0.8807, -0.2111,  0.5753],
         ...,
         [ 0.1896, -1.5526,  0.4400,  ..., -0.1049, -1.0098, -0.0797],
         [-1.4976,  0.1696, -1.0664,  ..., -0.1944, -0.7228, -0.8235],
         [-0.3761, -1.6515, -0.5306,  ...,  1.8987,  0.4523, -1.9342]]]), tensor([[[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 1., 0.,  ..., 0., 1., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 1.],
         [1., 1., 1.,  ..., 1., 0., 0.],
         [0., 0., 1.,  ..., 0., 1., 0.]]]))


## DataLoader

In [5]:
loader = DataLoader(dataset, batch_size=8, shuffle=True)
print(len(dataset))
for images, masks in loader:
    print(images.size())
    


100
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([8, 1, 64, 64])
torch.Size([4, 1, 64, 64])


## Building a Simple CNN

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.classifier(x)

model = SimpleCNN()
print(model)


SimpleCNN(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=1, bias=True)
    (2): Sigmoid()
  )
)


## Training Loop Example

In [7]:
model = SimpleCNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(2):
    for images, masks in loader:
        optimizer.zero_grad()
        preds = model(images)
        loss = criterion(preds, torch.zeros_like(preds))  # dummy target
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')


Epoch 1, Loss: 0.0000
Epoch 2, Loss: 0.0000


## U‑Net Building Blocks

In [8]:
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU()
    )

sample = torch.randn(1, 1, 128, 128)
block = conv_block(1, 16)
output = block(sample)
output.shape


torch.Size([1, 16, 128, 128])

## Exercises


1. Modify `RandomMRIDataset` to return a resized (128×128) slice using interpolation.  
2. Add another convolution layer to `SimpleCNN` and observe how the model size changes.  
3. Implement a small encoder-decoder network (mini U-Net) using `conv_block`.  
4. Write a custom Dice Loss function in PyTorch.  
5. Train the CNN on the random dataset and plot the loss curve using matplotlib.  


## Solutions
### 1. Resized Dataset (128x128)

In [9]:

class ResizedRandomMRIDataset(RandomMRIDataset):
    def __getitem__(self, idx):
        image, mask = super().__getitem__(idx)
        image = torch.nn.functional.interpolate(image.unsqueeze(0), size=(128,128), mode='bilinear', align_corners=False).squeeze(0)
        mask = torch.nn.functional.interpolate(mask.unsqueeze(0), size=(128,128), mode='nearest').squeeze(0)
        return image, mask


### 2. Deeper CNN

In [10]:

class DeeperCNN(SimpleCNN):
    def __init__(self):
        super().__init__()
        self.conv3 = nn.Conv2d(16, 16, kernel_size=3, padding=1)

    def forward(self, x):
        x = super().forward(x)
        return x


### 3. Mini U-Net

In [11]:

class MiniUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc1 = conv_block(1, 16)
        self.enc2 = conv_block(16, 32)
        self.pool = nn.MaxPool2d(2)
        self.dec1 = conv_block(32, 16)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.final = nn.Conv2d(16, 1, kernel_size=1)

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.up(x2)
        x4 = self.dec1(x3)
        return self.final(x4)


### 4. Dice Loss

In [12]:

class DiceLoss(nn.Module):
    def forward(self, preds, targets, smooth=1.0):
        preds = torch.sigmoid(preds)
        intersection = (preds * targets).sum()
        dice = (2. * intersection + smooth) / (preds.sum() + targets.sum() + smooth)
        return 1 - dice
